In [73]:
import pandas as pd

In [74]:
# Load the data
df = pd.read_csv('../1. Scrape/ads.csv')
df.head()

,link,title,address,area,age,rooms,total_price,price_per_meter,floor,elevator,parking,warehouse,longitude,latitude
0,https://divar.ir/v/-/wZmAzMs2,۹۵متر/ فول امکانات / فول بازسازی,ایران,95,1383,2,8000000000,84210000,3,True,True,True,51.444726,35.684725
1,https://divar.ir/v/-/wZl0tBJo,اپارتمان۵۰متر۱خواب نوسازتکواحدی گیلکی آهنگ مینا,مینا,50,1403,1,4400000000,88000000,3 از 4,True,True,False,51.449087,35.670433
2,https://divar.ir/v/-/wZmcALyN,۱۱۵ متر دوخواب شیک و...,یوسف‌آباد,115,1398,2,15890000000,138173000,4,True,True,True,NaN,NaN
3,https://divar.ir/v/-/wZgQZf-8,تکواحدی با پارکینگ,شهرک ولیعصر,51,1390,1,2000000000,39215000,2 از 4,False,True,True,51.345065,35.648787
4,https://divar.ir/v/-/wZXc6Q-z,نیاوران ۳۵۰ متر/سه جهت نور گیر/۴تراس,نیاوران,350,1390,4,52800000000,150857000,5,True,True,True,51.452451,35.812732


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9577 entries, 0 to 9576
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   link             9577 non-null   object 
 1   title            9577 non-null   object 
 2   address          9577 non-null   object 
 3   area             9577 non-null   int64  
 4   age              9577 non-null   object 
 5   rooms            9577 non-null   object 
 6   total_price      9577 non-null   int64  
 7   price_per_meter  9577 non-null   int64  
 8   floor            9577 non-null   object 
 9   elevator         9577 non-null   bool   
 10  parking          9577 non-null   bool   
 11  warehouse        9577 non-null   bool   
 12  longitude        7824 non-null   float64
 13  latitude         7824 non-null   float64
dtypes: bool(3), float64(2), int64(3), object(6)
memory usage: 851.2+ KB


In [76]:
df.describe()

,area,total_price,price_per_meter,longitude,latitude
count,9577.000000,9.577000e+03,9.577000e+03,7824.000000,7824.000000
mean,170.804845,1.317370e+10,1.053560e+08,51.382206,35.723658
std,6143.585901,2.497810e+10,7.131870e+07,0.127218,0.060613
min,1.000000,1.110000e+02,0.000000e+00,51.124653,32.435613
25%,63.000000,3.900000e+09,6.074000e+07,51.327518,35.692147
50%,88.000000,7.800000e+09,9.318100e+07,51.374981,35.729518
75%,122.000000,1.500000e+10,1.333330e+08,51.449528,35.758036
max,600000.000000,1.000000e+12,2.222222e+09,59.510125,36.317200


In [77]:
df.shape

(9577, 14)

In [78]:
# Check for missing values
df.isnull().sum()

link                  0
title                 0
address               0
area                  0
age                   0
rooms                 0
total_price           0
price_per_meter       0
floor                 0
elevator              0
parking               0
warehouse             0
longitude          1753
latitude           1753
dtype: int64

There are 1753 houses without a location specified. It will not affect the analysis, so we will keep them.

In [79]:
# Check for duplicates
df.duplicated().sum()

0

No duplicate rows found.

Map addresses to regions

In [80]:
regions = {
    1: ['نیاوران', 'ولنجک', 'زعفرانیه', 'ازگل', 'قیطریه', 'اراج', 'فرمانیه', 'سوهانک', 'محمودیه', 'شهرک نفت (منطقه ۱)', 'چیذر', 'دزاشیب', 'دربند', 'هزارسنگ', 'گلاب دره', 'دارآباد', 'حصار بوعلی', 'حکمت', 'شهرک محلاتی', 'باغ فردوس', 'درکه', 'تجریش', 'امامزاده قاسم', 'جماران', 'کاشانک', 'اوین', 'الهیه', 'کامرانیه', 'آجودانیه'],
    2: ['پونک', 'ستارخان', 'مرزداران', 'گیشا (کوی نصر)', 'صادقیه', 'شریف', 'سعادت‌آباد', 'سپهر', 'دریان‌نو', 'طرشت', 'توحید', 'شهرآرا', 'تهران‌ویلا', 'آسمان', 'ایوانک', 'شادمهر', 'شهرک ژاندارمری', 'شهرک غرب', 'دریا', 'دانشگاه شریف', 'پاتریس لومومبا', 'کوی فراز', 'پرواز'],
    3: ['دروس', 'امانیه', 'اختیاریه', 'قلهک', 'سیدخندان', 'ونک', 'زرگنده', 'قبا', 'میرداماد', 'درب دوم', 'جلفا', 'ظفر', 'جردن', 'ده‌ونک', 'شیرازی'],
    4: ['شهرک ولیعصر', 'تهرانپارس شرقی', 'مبارک‌آباد بهشتی', 'اوقاف', 'شیان', 'تهرانپارس غربی', 'دانشگاه علم و صنعت', 'حکیمیه', 'قاسم‌آباد', 'کاظم‌آباد', 'قنات‌کوثر', 'شمیران‌نو', 'لویزان', 'مجیدآباد', 'خاک سفید', 'حسین‌آباد', 'کوهک', 'پلیس', 'هروی', 'شمس‌آباد', 'جوادیه تهرانپارس', 'شهرک امید', 'اتحاد'],
    5: ['شهر زیبا', 'سازمان برنامه', 'جنت‌آباد جنوبی', 'باغ فیض', 'ارم', 'جنت‌آباد مرکزی', 'شهرک نفت (منطقه ۵)', 'اباذر', 'شهران شمالی', 'اندیشه (شهر زیبا)', 'جنت‌آباد شمالی', 'سازمان آب', 'المهدی', 'شهران جنوبی', 'حصارک', 'کوی بیمه', 'کن', 'بهاران', 'اکباتان', 'شهرک آپادانا', 'مرادآباد', 'کوی مهران', 'شهرک پرواز', 'شاهین', 'شهرک کوهسار'],
    6: ['امیرآباد', 'یوسف‌آباد', 'جمال‌زاده', 'سنایی', 'فاطمی', 'شهرک والفجر (منطقه ۶)', 'بلوار کشاورز', 'ایرانشهر', 'بهجت‌آباد', 'جهاد', 'گاندی', 'نجات اللهی', 'میدان ولیعصر', 'توانیر', 'قزل قلعه', 'نصرت', 'آرژانتین', 'دانشگاه تهران'],
    7: ['ارامنه', 'سهروردی', 'گرگان', 'خواجه نظام الملک', 'نظام‌آباد', 'نیلوفر', 'بهار', 'دبستان', 'حشمتیه', 'خواجه نصیر طوسی', 'امام حسین(ع)', 'عباس‌آباد', 'شارق شرقی', 'خاقانی'],
    8: ['دردشت', 'فدک', 'مدائن', 'هفت حوض', 'نارمک', 'سبلان', 'وحیدیه', 'تسلیحات', 'کرمان', 'مجیدیه', 'زرکش', 'لشکر', 'نارمک جنوبی'],
    9: ['دکتر هوشیار', 'مهرآباد جنوبی', 'استاد معین', 'شهید دستغیب', 'شمشیری', 'سرآسیاب مهرآباد', 'طوس'],
    10: ['سلسبیل', 'سلیمانی', 'بریانک', 'هفت چنار', 'جی', 'هاشمی', 'زنجان', 'نواب', 'آذربایجان', 'جیحون', 'شبیری'],
    11: ['اسکندری', 'سلامت', 'شهرک راه‌آهن', 'جمهوری', 'میدان انقلاب', 'امیریه', 'شیخ هادی', 'قلمستان', 'مخصوص', 'منیریه', 'میدان حر', 'حسن‌آباد', 'گمرک', 'امیر بهادر'],
    12: ['ایران', 'سنگلج', 'کوثر', 'دروازه شمیران', 'بهارستان', 'فردوسی', 'سیروس', 'بازار', 'قیام', 'مولوی', 'آبشار'],
    13: ['نیرو هوایی', 'پیروزی', 'دهقان', 'حافظیه', 'آشتیانی', 'زاهد گیلانی', 'امامت', 'صفا', 'تهران‌نو'],
    14: ['مینا', 'پرستار', 'شیوا', 'چهارصد دستگاه', 'دولاب', 'نبی اکرم(ص)', 'تاکسیرانی', 'دلگشا', 'فلاح', 'صد دستگاه', 'آهنگ', 'ائمه اطهار', 'شاهد', 'خاوران', 'شکوفه'],
    15: ['افسریه', 'مسعودیه', 'ابوذر (منطقه ۱۵)', 'شهرک کیانشهر', 'اتابک', 'شوش', 'مشیریه', 'بیسیم', 'قیام‌دشت', 'مینابی', 'هاشم‌آباد', 'طیب', 'کاروان', 'مسگرآباد', 'شریف‌آباد'],
    16: ['خزانه', 'علی‌آباد', 'یاخچی‌آباد', 'باغ آذری', 'نازی‌آباد', 'جوادیه', 'شهرک تختی'],
    17: ['بلورسازی', 'امام سجاد(ع)', 'امامزاده حسن(ع)', 'گلچین', 'زمزم', 'جلیلی', 'مقدم', 'زهتابی', 'باغ خزانه', 'آذری', 'وصفنارد'],
    18: ['خلیج فارس', 'شادآباد', 'بهداشت', 'تولید دارو', 'شهید رجایی', 'سعیدآباد', 'کوی هفدهم شهریور', 'یافت‌آباد', 'صاحب الزمان', 'کوی فردوس', 'شهرک مسلمین', 'شاندیز', 'شهرک فردوس', 'شهرک امام خمینی', 'ابراهیم‌آباد', 'شهرک طالقانی'],
    19: ['خانی‌آباد نو', 'عبدل‌آباد', 'شهرک شریعتی', 'دولتخواه', 'خانی‌آباد', 'نعمت‌آباد', 'بهمن یار', 'اسفندیاری', 'شهرک ابوذر', 'کریم‌آباد'],
    20: ['دولت‌آباد', 'سرتخت', 'اقدسیه', 'ظهیرآباد', 'استخر', 'دیلمان', 'شهید آوینی', 'فیروزآبادی', 'سیزده آبان', 'صفائیه (چشمه علی)', 'حمزه‌آباد', 'شهادت', 'جوانمرد قصاب', 'ابن بابویه'],
    21: ['چیتگر', 'تهران‌سر', 'پاسداران', 'زیبادشت', 'شهرک صدرا', 'وردآورد', 'دهکده المپیک', 'شهرک دریا', 'هوانیروز', 'شهرک استقلال', 'شهرک پاسداران', 'شهرک آزادی', 'شهرک دانشگاه تهران', 'شهرک سینمایی', 'شهرک دانشگاهی'],
    22: ['سرو آزاد', 'شهرک شهید باقری', 'دریاچه شهدای خلیج فارس', 'آبشار تهران'],
}

In [81]:
# invert the dictionary to map addresses to regions
address_to_region = {address: region for region, addresses in regions.items() for address in addresses}

In [82]:
# a new column for the region
df['region'] = df['address'].map(address_to_region)
df.head()

,link,title,address,area,age,rooms,total_price,price_per_meter,floor,elevator,parking,warehouse,longitude,latitude,region
0,https://divar.ir/v/-/wZmAzMs2,۹۵متر/ فول امکانات / فول بازسازی,ایران,95,1383,2,8000000000,84210000,3,True,True,True,51.444726,35.684725,12
1,https://divar.ir/v/-/wZl0tBJo,اپارتمان۵۰متر۱خواب نوسازتکواحدی گیلکی آهنگ مینا,مینا,50,1403,1,4400000000,88000000,3 از 4,True,True,False,51.449087,35.670433,14
2,https://divar.ir/v/-/wZmcALyN,۱۱۵ متر دوخواب شیک و...,یوسف‌آباد,115,1398,2,15890000000,138173000,4,True,True,True,NaN,NaN,6
3,https://divar.ir/v/-/wZgQZf-8,تکواحدی با پارکینگ,شهرک ولیعصر,51,1390,1,2000000000,39215000,2 از 4,False,True,True,51.345065,35.648787,4
4,https://divar.ir/v/-/wZXc6Q-z,نیاوران ۳۵۰ متر/سه جهت نور گیر/۴تراس,نیاوران,350,1390,4,52800000000,150857000,5,True,True,True,51.452451,35.812732,1


In [83]:
# put the new column in the right place
address_index = df.columns.get_loc('address')
df.insert(address_index + 1, 'region', df.pop('region'))
df.head()

,link,title,address,region,area,age,rooms,total_price,price_per_meter,floor,elevator,parking,warehouse,longitude,latitude
0,https://divar.ir/v/-/wZmAzMs2,۹۵متر/ فول امکانات / فول بازسازی,ایران,12,95,1383,2,8000000000,84210000,3,True,True,True,51.444726,35.684725
1,https://divar.ir/v/-/wZl0tBJo,اپارتمان۵۰متر۱خواب نوسازتکواحدی گیلکی آهنگ مینا,مینا,14,50,1403,1,4400000000,88000000,3 از 4,True,True,False,51.449087,35.670433
2,https://divar.ir/v/-/wZmcALyN,۱۱۵ متر دوخواب شیک و...,یوسف‌آباد,6,115,1398,2,15890000000,138173000,4,True,True,True,NaN,NaN
3,https://divar.ir/v/-/wZgQZf-8,تکواحدی با پارکینگ,شهرک ولیعصر,4,51,1390,1,2000000000,39215000,2 از 4,False,True,True,51.345065,35.648787
4,https://divar.ir/v/-/wZXc6Q-z,نیاوران ۳۵۰ متر/سه جهت نور گیر/۴تراس,نیاوران,1,350,1390,4,52800000000,150857000,5,True,True,True,51.452451,35.812732


In [84]:
# unique regions
df['region'].unique()

array([12, 14,  6,  4,  1,  3,  2,  5, 10, 18,  8,  9, 15,  7, 22, 21, 11,
       13, 19, 16, 20, 17])

All set up!

In [85]:
df['floor'].unique()

array(['3', '3 از 4', '4', '2 از 4', '5', '1 از 4', '3 از 11', '12 از 14',
       '5 از 9', '5 از 5', '3 از 5', '9 از 11', '4 از 11', '4 از 4',
       '4 از 5', 'همکف از 4', '1', '1 از 3', '6', '15 از 12', '2 از 6',
       '3 از 3', 'زیرهمکف از 4', '4 از 6', '2 از 3', '8 از 2', '2 از 5',
       '12 از بیشتر از 30', '1 از 5', '6 از 6', '8 از 14', '2', '1 از 10',
       '2 از 8', '1 از 6', 'همکف', '7 از 30', '3 از 6', '12',
       '11 از بیشتر از 30', '3 از 8', '8 از 30', '9', '13', '5 از 11',
       '7 از 11', '7 از 7', '9 از 17', '8 از 8', '1 از 19', '11 از 12',
       'همکف از 5', '5 از 6', '8', '17', '7 از 10', '8 از 11', '10 از 14',
       '15', '2 از 2', '+30 از بیشتر از 30', '2 از 10',
       '1 از بیشتر از 30', '+30', '1 از 7', '6 از 7', '16', '10 از 15',
       '15 از 18', '12 از 21', '4 از 7', '8 از 15', '12 از 12',
       '11 از 11', '18 از 29', '9 از بیشتر از 30', '25 از 25', '17 از 25',
       '8 از 18', '5 از 13', '12 از 15', '7', '9 از 14', '20', '4 از 9',
       '7 از 19'

In [86]:
import numpy as np

def normalize_floor(value):
    if value == '+30':
        return 31
    if pd.isna(value):  # Handle NaN
        return np.nan
    if 'همکف' in value:  # Ground floor
        return 0
    if 'زیرهمکف' in value:  # Below ground
        return -1
    if 'از' in value:  # Extract the number before "از"
        return int(value.split('از')[0].strip())
    try:  # Try converting to integer
        return int(value.strip())
    except ValueError:
        return np.nan
    
df['floor'] = df['floor'].apply(normalize_floor)
df['floor'].unique()

array([ 3,  4,  2,  5,  1, 12,  9,  0,  6, 15,  8,  7, 11, 13, 17, 10, 30,
       31, 16, 18, 25, 20, 28, 14, 24, 21, 23, 26, 19, 22, 27, 29])

In [87]:
df['floor'].value_counts()

floor
3     2024
2     2018
1     1775
4     1565
5      938
0      292
6      257
10     109
7      105
8      102
9       66
12      59
11      36
14      34
15      33
18      22
20      22
13      21
16      20
17      19
19      12
30      10
22       5
25       5
26       4
31       4
24       4
21       4
23       4
27       3
29       3
28       2
Name: count, dtype: int64

In [88]:
df['age'].unique()

array(['1383', '1403', '1398', '1390', 'قبل از 1370', '1402', '1385',
       '1388', '1401', '1375', '1392', '1386', '1381', '1373', '1400',
       '1397', '1394', '1380', '1395', '1389', '1382', '1396', '1384',
       '1393', '1387', '1374', '1376', '1391', '1399', '1378', '1379',
       '1371', '1372', '1377'], dtype=object)

In [89]:
import re

df['age'] = df['age'].apply(lambda x: x.split()[-1].strip() if 'از' in x else x)
df['age'] = df['age'].apply(lambda x: int(x) if re.match(r'^\d+$', str(x)) else 0)
df['age'].unique()

array([1383, 1403, 1398, 1390, 1370, 1402, 1385, 1388, 1401, 1375, 1392,
       1386, 1381, 1373, 1400, 1397, 1394, 1380, 1395, 1389, 1382, 1396,
       1384, 1393, 1387, 1374, 1376, 1391, 1399, 1378, 1379, 1371, 1372,
       1377])

In [90]:
df['rooms'].unique()

array(['2', '1', '4', '3', '+4', 'بدون اتاق'], dtype=object)

In [91]:
df['rooms'].value_counts()

rooms
2            4752
1            2427
3            2177
4             182
+4             23
بدون اتاق      16
Name: count, dtype: int64

In [92]:
room_map = {'+4': 5, 'بدون اتاق': 0}
df['rooms'] = df['rooms'].map(room_map).fillna(df['rooms'])
df['rooms'] = df['rooms'].apply(lambda x: int(x))
df['rooms'].value_counts()

rooms
2    4752
1    2427
3    2177
4     182
5      23
0      16
Name: count, dtype: int64

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9577 entries, 0 to 9576
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   link             9577 non-null   object 
 1   title            9577 non-null   object 
 2   address          9577 non-null   object 
 3   region           9577 non-null   int64  
 4   area             9577 non-null   int64  
 5   age              9577 non-null   int64  
 6   rooms            9577 non-null   int64  
 7   total_price      9577 non-null   int64  
 8   price_per_meter  9577 non-null   int64  
 9   floor            9577 non-null   int64  
 10  elevator         9577 non-null   bool   
 11  parking          9577 non-null   bool   
 12  warehouse        9577 non-null   bool   
 13  longitude        7824 non-null   float64
 14  latitude         7824 non-null   float64
dtypes: bool(3), float64(2), int64(7), object(3)
memory usage: 926.0+ KB


In [97]:
df.describe()

,region,area,age,rooms,total_price,price_per_meter,floor,longitude,latitude
count,9577.000000,9577.000000,9577.000000,9577.000000,9.577000e+03,9.577000e+03,9577.000000,7824.000000,7824.000000
mean,8.060875,170.804845,1393.206432,2.015767,1.317370e+10,1.053560e+08,3.437820,51.382206,35.723658
std,6.256338,6143.585901,8.224343,0.764761,2.497810e+10,7.131870e+07,3.157208,0.127218,0.060613
min,1.000000,1.000000,1370.000000,0.000000,1.110000e+02,0.000000e+00,0.000000,51.124653,32.435613
25%,3.000000,63.000000,1387.000000,1.000000,3.900000e+09,6.074000e+07,2.000000,51.327518,35.692147
50%,6.000000,88.000000,1394.000000,2.000000,7.800000e+09,9.318100e+07,3.000000,51.374981,35.729518
75%,11.000000,122.000000,1401.000000,2.000000,1.500000e+10,1.333330e+08,4.000000,51.449528,35.758036
max,22.000000,600000.000000,1403.000000,5.000000,1.000000e+12,2.222222e+09,31.000000,59.510125,36.317200


In [99]:
df.isnull().sum()

link                  0
title                 0
address               0
region                0
area                  0
age                   0
rooms                 0
total_price           0
price_per_meter       0
floor                 0
elevator              0
parking               0
warehouse             0
longitude          1753
latitude           1753
dtype: int64

In [100]:
df.sample(5)

,link,title,address,region,area,age,rooms,total_price,price_per_meter,floor,elevator,parking,warehouse,longitude,latitude
3397,https://divar.ir/v/-/wZhYdy1i,پونک/۹۲ متر نوساز آماده تحویل/همیلا پارک,پونک,2,92,1403,2,16700000000,181521000,4,True,True,True,51.339742,35.759796
4244,https://divar.ir/v/-/wZlsdtsq,۱۲۰متر/چشم انداز ابدی شمال طهران/۲تراس اختصاصی,هروی,4,120,1402,2,14400000000,120000000,3,True,True,True,NaN,NaN
228,https://divar.ir/v/-/wZmEkJgL,۹۲ متر / ۲ خواب / شهران شمالی / فول دیزاین,شهران شمالی,5,92,1393,2,10300000000,111956000,5,True,True,True,51.288683,35.765600
5986,https://divar.ir/v/-/wZOMGruH,آپارتمان ۱۰۸ متری کلید نخورده خ شیرمحمدی,نازی‌آباد,16,108,1403,2,8856000000,82000000,3,True,True,False,51.393905,35.640140
379,https://divar.ir/v/-/wZm8Tyjr,۷۱متری هفت چنار کلیدنخورده فول امکانات,هفت چنار,10,71,1403,2,4650000000,65492000,4,True,True,True,51.369886,35.671614


Finally! The data is clean and ready for analysis.

In [96]:
df.to_csv('cleaned_ads.csv', index=False)